In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchio as tio
import torchvision.transforms as transforms

from modules.Dataset import *
from modules import Evaluator3D, models
from modules.LossFunctions import DC_and_CE_loss, GDiceLossV2
from modules.Predictor import Predictor
from modules.Transforms import *
from modules.Utils import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# There are multiple data folders belong to same dataset. Each of them processed in different ways.
# Therefore, path of the data and its name explicitly are defined.
dataset_path = "Data/rel3_dhcp_anat_pipeline//"
cv_ = "cv3" # 5-fold cross-validation. Folds [cv1-cv5]

# Transformations.
transform_eval = None #transforms.Compose([RandomMotion(), RandomAffine(degrees=[15])])


# Split dataset.
train = MRIDataset(LateWeeks, "train", dataset_path, transform=transform_eval)
val = MRIDataset(LateWeeks, "val", dataset_path, transform=transform_eval)
test = MRIDataset(LateWeeks, "test", dataset_path, transform=transform_eval)

torch.manual_seed(0)
val_loader = torch.utils.data.DataLoader(dataset=val, batch_size=1)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=1)

In [ ]:
model = models.UNet3D().to(device)
criterion = DC_and_CE_loss({'batch_dice': True, 'smooth': 1e-5, 'do_bg': False, 'square': False}, {})
model.load_state_dict(torch.load("output/UNet3D/Iteration_20222812/run_dHCP/weights/36_model.pth"))

In [ ]:
for i in range(len(val)):
    sub = val[i]
    predictor = Predictor(model, (128, 128, 128))
    output = predictor.predict(sub)
    output = output.argmax(dim=1)
    save_nii('Predictions/UNet3D/model_40/', 
             f'{sub.sub_id}_pred', 
             output.squeeze(0).numpy().astype(np.float64))

In [ ]:
for i in range(len(test)):
    sub = test[i]
    predictor = Predictor(model, (128, 128, 128))
    output = predictor.predict(sub)
    output = output.argmax(dim=1)
    save_nii('Predictions/UNet3D/model_40/', 
             f'{sub.sub_id}_pred', 
             output.squeeze(0).numpy().astype(np.float64))

In [ ]:
train[6].sub_id

In [ ]:
sub = train[6]
predictor = Predictor(model, (128, 128, 128))
output = predictor.predict(sub)
predictor.plot_slice(output, sub.mask.data, 'sagittal', 45)

In [ ]:
output = output.argmax(dim=1)

In [ ]:
save_nii('Predictions/UNet3D', 
         f'sub-049_pred', 
         output.squeeze(0).numpy().astype(np.float64), sub.mri.affine)

In [ ]:
sub = train[13]
predictor = Predictor(model, (128, 128, 128))
output = predictor.predict(sub)
output = output.argmax(dim=1)
save_nii('Predictions/UNet3D', 
         f'iter11_20221106_1404_{sub.sub_id}_pred', 
         output.squeeze(0).numpy().astype(np.float64))

In [ ]:
sub = test[0]

In [ ]:
sub_023 = nib.load('Predictions/UNet3D/model_40/sub-023_pred.nii.gz')

In [ ]:
np.unique(sub_023.get_fdata())

In [ ]:
sub.mask.data.unique()